# 라이브러리 import 및 경로 지정

In [ ]:
import warnings #경고 메시지 무시
warnings.filterwarnings(action='ignore')

import pandas as pd 
import pickle 
import json
import re #정규표현식 사용

from tqdm import tqdm #진행표시바
from konlpy.tag import Komoran #Kmoran 형태소 분석기 사용
komoran=Komoran()

import itertools
from collections import Counter

%matplotlib inline #브라우저 내부 시각화
import matplotlib
from IPython.display import set_matplotlib_formats
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from gensim import corpora, models

import gensim
from gensim.models import CoherenceModel #LDA 평가 지표

import pyLDAvis.gensim_models #LDA 시각화

from gensim.models import Word2Vec
from sklearn.manifold import TSNE

In [ ]:
path = '' #파일 경로 지정
font_path = path + '' #폰트 경로 지정

In [ ]:
raw_data = pd.read_excel(path + 'rawdata.xlsx', engine="openpyxl") #판다스의 기본 engine이 xlrd로 설정되어 있는 경우 오류 대비

In [ ]:
raw_data.info()

In [ ]:
#raw_data.groupby(['ch','ch2']).size()

# 데이터 전처리

### 홍보, 광고, 스팸 목적성 문서 제거

In [ ]:
#제거할 문서의 기준이 되는 특정 단어 불러오기
f = open(path + 'row_del.txt', 'r', encoding='UTF-8')
remove = list(map(lambda x: x[:-1], f.readlines())) #읽어올때 개행문자 제거
f.close()
#remove

In [ ]:
#특정 단어 포함 문서 제거  #홍보나 광고성 목적의 문서 제거
indx = []
for i in range(len(raw_data)):
    for j in range(len(remove)):
        if remove[j] in raw_data['document'][i] or remove[j] in raw_data['title'][i]:  #document나 title에 포함되는 문서 제거
            indx.append(i)
set_indx = list(dict.fromkeys(indx)) #리스트 순서 유지하면서 중복 인덱스 제거  #python 3.7버전부터 딕셔너리가 삽입 순서 보존

In [ ]:
pre_data = raw_data.drop(raw_data.index[set_indx])  #해당 인덱스 제거
pre_data.reset_index(drop=True, inplace=True)  #데이터프레임 인덱스 리셋
# pre_data  #출력

### 한글 외 문자 제거

In [ ]:
#문서 내용 한글만 남기고 제거
pre_data['pre_doc'] = pre_data['document'].apply(lambda x: re.sub("[^가-힣]", " ", x))
# pre_data  #출력

In [ ]:
#문서 내용이 두글자 미만이나 공백인 경우 제거
indx = []
for i in range(len(pre_data)):
    if (len(pre_data['pre_doc'][i]) < 2 or pre_data['pre_doc'][i].isspace() == True):
        indx.append(i)

In [ ]:
pre_data = pre_data.drop(pre_data.index[indx])  #해당 인덱스 제거
pre_data.reset_index(drop=True, inplace=True)  #데이터프레임 인덱스 리셋
# pre_data  #출력

### 형태소 분석 및 명사 추출

In [ ]:
docs_n=[]
for doc in tqdm(pre_data['pre_doc']):
    doc_n = list(term for term in komoran.nouns(doc) if len(term)>1) #형태소 분석_2글자 이상의 명사 추출  #for문 내 if문 구조
    docs_n.append(doc_n) #이모티콘 등의 특수문자를 처리하는 경우, try except UnicodeDecodeError문 추가
#docs_n   #출력

### 불용어 제거

In [ ]:
#불용어 사전 읽어오기
f = open(path + 'stopwords-ko.txt', 'r', encoding='UTF-8')
stopwords = list(map(lambda x: x[:-1], f.readlines())) #읽어올때 개행문자 제거
f.close()
# stopwords   #출력

In [ ]:
#추출한 명사에서 불용어 제거
for doc_n in docs_n:
    for word in stopwords:
        while word in doc_n:
            doc_n.remove(word)

pre_data['doc_noun'] = docs_n  #데이터프레임에 삽입
#pre_data   #출력

### 저장/읽기

In [ ]:
# pickle 파일 저장하는 함수
def save(data, name):
    with open(path+ f'{name}.pickle', 'wb') as f: 
        pickle.dump(data, f)

In [ ]:
# pickle 파일 불러오는 함수
def load(name):
    with open(path + f'{name}.pickle','rb') as fr:
        data = pickle.load(fr)
    return data

In [ ]:
save(pre_data, 'pre_data')

In [ ]:
pre_data = load('pre_data')

# 토픽 모델링

### corpus 생성

In [ ]:
noun_dic = corpora.Dictionary(docs_n) #명사 리스트를 바탕으로 단어 빈도별 목록 생성 (토큰화)
noun_dic.filter_extremes(no_below=3, no_above=0.9) # 빈도가 3 미만이거나 전체의 90% 이상인 단어 제외

In [ ]:
corpus = [noun_dic.doc2bow(doc_n) for doc_n in docs_n] #토픽 모델링을 위한 DTM(문서단어행렬)을 생성 (벡터화)
                                                        #doc2bow : 문서를 단어의 id와 빈도수로 수치화

### 토픽 수에 따른 성능 평가

In [ ]:
#토픽수에 따른 혼잡도와 일관성 분석 후, 최선의 토픽수 선정
Lda = gensim.models.ldamodel.LdaModel  #토픽모델링 기법: 1.LDA(확률을 바탕으로 단어가 특정 주제에 존재할 확률과 문서에 특정 주제가 존재할 확률을 결합확률로 추정하여 토픽추출) / 2.LSA(분절된 단어들에 벡터값을 부여하고 차원축소를 하여 축소된 차원에서 근접한 단어들을 주제로 묶음)
perplexity_score=[]
coherence_score=[]

for i in range(1,10): #토픽수가 1~ 9일때 혼잡도와 일관성을 측정
    ldamodel=Lda(corpus, num_topics=i, id2word=noun_dic, passes=15, iterations=200, random_state=0)  #passes: 모델 학습시 전체 코퍼스에서 모델을 학습시키는 빈도  #iterations: 각 문서 반복 빈도
    perplexity_score.append(ldamodel.log_perplexity(corpus)) 
    coherence_score.append(CoherenceModel(model=ldamodel, corpus=corpus, coherence='u_mass').get_coherence())
    print(i, 'process complete')

In [ ]:
plt.plot(range(1,10),perplexity_score,'r',marker='^') #혼잡도 시각화
plt.xlabel("number of topics")
plt.ylabel("perplexity score")
plt.show()

In [ ]:
plt.plot(range(1,10),coherence_score,'b',marker='o') #일관성 시각화
plt.xlabel("number of topics")
plt.ylabel("coherence score")
plt.show()

### LDA 토픽 모델링

In [ ]:
#지정 토픽수로 토픽 모델링 진행
lda_model = Lda(corpus, num_topics=4, id2word=noun_dic, passes=15, iterations=200, random_state=0)

#토픽별 5 단어씩 출력
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

### 시각화

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, noun_dic)
pyLDAvis.display(vis)

In [ ]:
pyLDAvis.save_html(vis, path + 'ldamodel.html') #토픽모델링 결과를 html로 저장

### 데이터프레임에 토픽 삽입

In [ ]:
topics = []
for i in range(len(corpus)):
    prop_sort=[]
    topic_sort=[]
    for topic , prop in lda_model.get_document_topics(corpus)[i]:
        prop_sort.append(prop)
        topic_sort.append(topic)
    topics.append(topic_sort[prop_sort.index(max(prop_sort))]) #각 문서를 가장 큰 확률을 가진 토픽에 배정

In [ ]:
total_docs = pre_data[['pre_doc','doc_noun']]
total_docs['topic'] = topics
total_docs['topic'] = total_docs['topic'].apply(lambda x: x+1) #topic이 0부터 시작하는 것을 1부터 시작으로 변경
# total_docs   #출력

### 저장/읽기

In [ ]:
save(total_docs, 'topic_data')

In [ ]:
total_docs = load('topic_data')

# 토픽별 주요 키워드 시각화

## 1. 워드클라우드 (빈도 기반)

In [ ]:
#상위 빈도 단어 50개를 추출하는 함수
def noun_list(cate):
    if cate == 'total':
        df = total_docs
    else:
        df = total_docs[total_docs['topic']==cate]
        
    noun_list = list(itertools.chain(*list(df['doc_noun'])))
    count = Counter(noun_list)
    print(len(count)) #단어 종류 수
    fift = dict(count.most_common(50))
    return fift

In [ ]:
#워드클라우드로 시각화하는 함수
def fwordcloud(cate):  #category: 'total', 1, 2, 3, 4
    fift = noun_list(cate)
    matplotlib.rc('font', family='Malgun Gothic')
    set_matplotlib_formats('retina')  #한글 선명하게
    wordcloud = WordCloud(font_path=font_path, background_color='white',colormap="Accent_r", width=1500, height=1000).generate_from_frequencies(fift) 

    if cate == 'total':
        plt.imshow(wordcloud)
        plt.axis('off')
        plt.savefig(path + 'total.png') #이미지 저장
    else:
        plt.imshow(wordcloud)
        plt.axis('off')
        plt.savefig(path + f'topic{cate}.png') #이미지 저장

### 시각화

In [ ]:
fwordcloud('total') #전체 단어 시각화

In [ ]:
fwordcloud(1) #토픽1 단어 시각화

In [ ]:
fwordcloud(2) #토픽2 단어 시각화

In [ ]:
fwordcloud(3) #토픽3 단어 시각화

In [ ]:
fwordcloud(4) #토픽4 단어 시각화

## 2. Word2Vec, TSNE (유사도 기반)

In [ ]:
#w2v을 이용하여 단어를 유사도로 벡터화하는 함수
def w2v(topic):
    df = total_docs[total_docs['topic']==topic]
    model = Word2Vec(sentences = df['doc_noun'], size=50, window = 15, min_count=100, workers=4, iter=100, sg=1)
    word_vectors = model.wv.vectors # 어휘의 feature vector
    topic_w2v = (model, word_vectors)
    return topic_w2v

In [ ]:
topic1_w2v = w2v(1)
topic2_w2v = w2v(2)
topic3_w2v = w2v(3)
topic4_w2v = w2v(4)

In [ ]:
#tsne를 이용하여 2차원으로 시각화 (단어간 유사할수록 밀집되어 있음)
def tsne(w2v):
    vocab = list(w2v[0].wv.vocab)
    X = w2v[0][vocab]
    tsne = TSNE(n_components=2, random_state = 3, learning_rate = 500)
    X_tsne = tsne.fit_transform(X)
    df_plot = pd.DataFrame(X_tsne, index=vocab, columns=["x", "y"])
    
    fig = plt.figure()
    fig.set_size_inches(10, 10)
    ax = fig.add_subplot(1, 1, 1)

    ax.scatter(df_plot['x'], df_plot['y'])
    
    for word, pos in df_plot.iterrows():
        ax.annotate(word, pos)
    plt.show()

### 시각화

In [ ]:
tsne(topic1_w2v)  #토픽1 단어 시각화

In [ ]:
tsne(topic2_w2v) #토픽2 단어 시각화

In [ ]:
tsne(topic3_w2v) #토픽3 단어 시각화

In [ ]:
tsne(topic4_w2v) #토픽4 단어 시각화

# 긍부정 분석

### 각 문서별 긍부정 점수 계산

In [ ]:
def load_json(name):
    with open(path + f'{name}.json', encoding='UTF-8') as fr:
        data = json.load(fr)
        json_data = pd.DataFrame(data)
    return json_data

In [ ]:
s = load_json('Sentiword_info')
# s   #출력

In [ ]:
s_word = []  #감성사전 단어와 일치하는 단어
values = []  #문서에 부여된 값들
score = []   #문서 평균 점수

def average(list): #각 문서별 평균 긍부정 점수를 계산하는 함수
    return sum(list)/len(list)

for word in tqdm(total_docs['pre_doc']):
    temp_s_word=[]
    temp_value=[]
    for i in range(len(s)):
        if s.iloc[i]['word'] in word and len(s.iloc[i]['word']) > 1:  #감성사전 단어가 2글자 이상이며 문서 내 존재하는 경우, 점수 계산
            temp_s_word.append(s.iloc[i]['word'])
            temp_value.append(int(s.iloc[i]['polarity']))
    s_word.append(temp_s_word)
    values.append(temp_value)
    try:
        score.append(average(temp_value))
    except ZeroDivisionError:  #평균 점수를 계산할 값이 없는 경우  #0을 나누려할 때 나타남
        score.append(int(0))

In [ ]:
total_docs=total_docs.assign(sentiword=s_word, values=values, score=score)  #데이터프레임에 삽입
# total_docs   #출력

### 저장/읽기

In [ ]:
save(total_docs, 'senti_data')

In [ ]:
total_docs = load('senti_data')

### 데이터프레임에 긍부정 삽입

In [ ]:
senti = []
for i in range(len(total_docs)):
    if total_docs['score'][i] > 0:    #평균 점수>0 :긍정
        senti.append('긍정')
    elif total_docs['score'][i] < 0:  #평균 점수<0 :부정
        senti.append('부정')
    else:                             #평균 점수=0 :중립
        senti.append('중립') 

In [ ]:
total_docs['senti'] = senti
# total_docs   #출력

### 각 토픽별 긍부정 비율 확인

In [ ]:
#각 토픽별 긍부정 문서 비율을 계산하는 함수
def senti_cnt(topic):
    df = total_docs[total_docs['topic']==topic]
    pos = len(df[df['senti']=='긍정'])
    neu = len(df[df['senti']=='중립'])
    neg = len(df[df['senti']=='부정'])
    return [pos, neu, neg]

In [ ]:
# senti_cnt(1)  #출력  #토픽넘버에 해당하는 긍부정 문서 비율

In [ ]:
senti_bar = pd.DataFrame([senti_cnt(1),senti_cnt(2),senti_cnt(3),senti_cnt(4)],
                  index=['topic1','topic2','topic3','topic4'],
                  columns=['긍정','중립','부정'])
# senti_bar

### 시각화

In [ ]:
#막대 그래프 시각화
senti_plot = senti_bar.plot(kind='bar',
                            color=['dimgray', 'darkgray','lightgray'],
                           figsize=(11,7), rot=0, width = 0.85)

le=[]
he=[]
for p in senti_plot.patches:
    left, bottom, width, height = p.get_bbox().bounds 
    le.append(left)   #막대 그래프 위치 x값
    he.append(height) #막대 그래프 위치 y값

In [ ]:
#그래프에 글자 추가
senti_plot = senti_bar.plot(kind='bar',
                            color=['dimgray', 'darkgray','lightgray'],
                           figsize=(11,7), rot=0, width = 0.85)

# topic1 긍부정 비율
senti_plot.annotate("%.2f%%"%( senti_cnt(1)[0]/sum(senti_cnt(1)) ), (le[0]+0.3/2, he[0]*1.02), ha='center',fontsize=14)
senti_plot.annotate("%.2f%%"%( senti_cnt(1)[1]/sum(senti_cnt(1)) ), (le[4]+0.3/2, he[4]*1.02), ha='center',fontsize=14)
senti_plot.annotate("%.2f%%"%( senti_cnt(1)[2]/sum(senti_cnt(1)) ), (le[8]+0.3/2, he[8]*1.02), ha='center',fontsize=14)

# topic2 긍부정 비율
senti_plot.annotate("%.2f%%"%( senti_cnt(2)[0]/sum(senti_cnt(2)) ), (le[1]+0.3/2, he[1]*1.02), ha='center',fontsize=14)
senti_plot.annotate("%.2f%%"%( senti_cnt(2)[1]/sum(senti_cnt(2)) ), (le[5]+0.3/2, he[5]*1.02), ha='center',fontsize=14)
senti_plot.annotate("%.2f%%"%( senti_cnt(2)[2]/sum(senti_cnt(2)) ), (le[9]+0.3/2, he[9]*1.02), ha='center',fontsize=14)

# topic3 긍부정 비율
senti_plot.annotate("%.2f%%"%( senti_cnt(3)[0]/sum(senti_cnt(3)) ), (le[2]+0.3/2, he[2]*1.02), ha='center',fontsize=14)
senti_plot.annotate("%.2f%%"%( senti_cnt(3)[1]/sum(senti_cnt(3)) ), (le[6]+0.3/2, he[6]*1.02), ha='center',fontsize=14)
senti_plot.annotate("%.2f%%"%( senti_cnt(3)[2]/sum(senti_cnt(3)) ), (le[10]+0.3/2, he[10]*1.02), ha='center',fontsize=14)

# topic4 긍부정 비율
senti_plot.annotate("%.2f%%"%( senti_cnt(4)[0]/sum(senti_cnt(4)) ), (le[3]+0.3/2, he[3]*1.02), ha='center',fontsize=14)
senti_plot.annotate("%.2f%%"%( senti_cnt(4)[1]/sum(senti_cnt(4)) ), (le[7]+0.3/2, he[7]*1.02), ha='center',fontsize=14)
senti_plot.annotate("%.2f%%"%( senti_cnt(4)[2]/sum(senti_cnt(4)) ), (le[11]+0.3/2, he[11]*1.02), ha='center',fontsize=14)

senti_plot.legend(fontsize=13,loc=2)  # label 글자 크기, 위치 조정